In [1]:
import nbimporter
import helper_methods as hm
import preprocessing as pp
import numpy as np
import time
from sklearn import svm
import matplotlib.pyplot as plt
import cvxopt

Importing Jupyter notebook from helper_methods.ipynb
Importing Jupyter notebook from preprocessing.ipynb


### SVM Classifier Implementation
http://tullo.ch/articles/svm-py/  
https://pythonprogramming.net/soft-margin-kernel-cvxopt-svm-machine-learning-tutorial/  
https://xavierbourretsicotte.github.io/SVM_implementation.html  
http://www.cs.cmu.edu/~guestrin/Class/10701-S07/Slides/kernels.pdf  
https://courses.csail.mit.edu/6.867/wiki/images/a/a7/Qp-cvxopt.pdf  

In [2]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

In [3]:
# This function imitates a Polynomial Kernel with the dimension d as input to the function
def polynomial_kernel(x1, x2, degree = 3):
    return (1 + np.dot(x1, x2)) ** degree

In [4]:
def rbf_kernel(x1, x2, gamma):
    if gamma == 'auto':
        gamma = 1 / len(x1)
    return np.exp(gamma * (-np.linalg.norm(x1 - x2) ** 2))

In [5]:
class Support_Vector_Machine_Classifier:
    def __init__(self, C = 1.0, kernel = 'rbf', degree = 3, gamma = 'auto'):
        # https://stackoverflow.com/questions/60208/replacements-for-switch-statement-in-python
        self._kernel = {
                        'linear': linear_kernel,
                        'poly': lambda x1, x2: polynomial_kernel(x1, x2, degree = degree),
                        'rbf': lambda x1, x2: rbf_kernel(x1, x2, gamma = gamma),
                    }.get(kernel, rbf_kernel) 
        # to force the float type
        self._C = float(C)
        self._degree = degree
        self._gamma = gamma
        print('C used:', C)
        
    # https://stackoverflow.com/questions/1547145/defining-private-module-functions-in-python
    def _get_gram_matrix(self, X):
        n_samples, n_features = X.shape
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self._kernel(X[i], X[j])
        return K  
        
    
    def fit(self, X_train, Y_train):
        n_samples, n_features = X_train.shape
        
        K = self._get_gram_matrix(X_train)
        
        # We want to maximise wrt alpha, instead we minimise the negative of the expression wrt alpha
        # We want to maximise Q(a)
        # Q(a) = -(1/2)a(T)Pa + sum_i(a_i)
        # Instead we minimise (-Q(a))
        # Check SVM slide 14 and 20
        
        # P = sum i,j(yi * yj * K(xi, xj))
        P = cvxopt.matrix(np.outer(Y_train, Y_train) * K)
        # q = -1
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        
        # Ax = b ====> sum_i(y_i * a_i) = 0
        A = cvxopt.matrix(Y_train, (1, n_samples))
        b = cvxopt.matrix(0.0)
        
        # Gx <= h =====> 0 <= a <= C
        # Top half -----> -a <= 0
        top_half = np.eye(n_samples) * -1
        # Bottom half -----> a <= C
        bot_half = np.eye(n_samples)
        G = cvxopt.matrix(np.vstack((top_half, bot_half)))
        
        top_half = np.zeros(n_samples)
        bot_half = np.ones(n_samples) * self._C
        h = cvxopt.matrix(np.hstack((top_half, bot_half)))
        
        # Silent cvxopt solver - https://stackoverflow.com/a/27184422/6001624
        cvxopt.solvers.options['show_progress'] = True
        
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers (the alphas)
        alphas = np.ravel(solution['x'])

        # Support vectors have non zero lagrange multipliers (Check slide 22 SVM)
        is_sv = alphas > 1e-5
        sv_indices = np.arange(len(alphas))[is_sv]
        self._alphas = alphas[is_sv]
        
        # the x part of the SVs
        self._support_vectors = X_train[is_sv]
        # the y part of the SVs
        self._support_vector_labels = Y_train[is_sv]
        print("%d support vectors out of %d points" % (len(self._alphas), n_samples))

        # Slide 12: http://www.cs.cmu.edu/~guestrin/Class/10701-S07/Slides/kernels.pdf
        # Intercept -----> b = mean(y_k - sum_i(a_i * y_i * x_i).x_k) for all k st 0 < a_k < C
        #                    = mean(y_k - sum(a_i * y_i * K(x_i, x_k))) for all k st 0 < a_k < C
        self._b = 0
        for i in range(len(self._alphas)):
            self._b += self._support_vector_labels[i]
            self._b -= np.sum(self._alphas * self._support_vector_labels * K[sv_indices[i],is_sv]) # use is_sv as only those k st 0 < a_k < C
        self._b /= len(self._alphas)

        
    def predict(self, X):
        y_predict = np.ones(len(X)) * self._b
        for i in range(len(X)):
            for alpha, sv_y, sv in zip(self._alphas, self._support_vector_labels, self._support_vectors):
                y_predict[i] += alpha * sv_y * self._kernel(X[i], sv)
        return np.sign(y_predict)

### Finding Optimal Parameters
https://stats.stackexchange.com/questions/43943/which-search-range-for-determining-svm-optimal-c-and-gamma-parameters

In [6]:
def print_parameters_accuracy(accuracies):
    print('#Features \t C \t Gamma \t Accuracy')
    for i in range(len(accuracies)):
        print(accuracies[i][0], '\t\t', accuracies[i][1], '\t\t', accuracies[i][2], '\t\t', accuracies[i][3])
    print()

In [7]:
def find_optimal_values(max_features, C_list, gamma_list, kernel = 'rbf', num_splits = 10, symbol_name = 'AAPL', use_implementation = True):
    accuracies = list()
    for num_features in range(1, max_features + 1, 1):
        print('Features:', num_features)
        
        X_train, X_test, Y_train, Y_test = hm.prepare_data(num_features, symbol_name, is_binary_ouput=True)
        X_train, X_test, Y_train, Y_test = X_train.values, X_test.values, Y_train.values, Y_test.values
        
        for C in C_list: # [1, 10, 100, 500, 1000, 5000, 10000]:
            for gamma in gamma_list: # [1, 0.1, 0.05, 0.01, 0.005, 0.001]:
                print('(C, gamma) ------------------------> (' + str(C) + ', ' + str(gamma) + ')')
                skl_svm_tscv = svm.SVC(kernel=kernel, C=C, gamma=gamma)
#                 svm_tscv = Support_Vector_Machine_Classifier(C = C, kernel = kernel, gamma = gamma)
                if use_implementation:
                    accuracy = hm.timeSeriesCV(X_train, Y_train, num_splits, skl_svm_tscv, is_classification=True)
                else:
                    accuracy = hm.rolling_cross_validation(X_train, Y_train, num_splits, svm_tscv, is_classification=True)
                accuracies.append([num_features, C, gamma, accuracy])
    
    print_parameters_accuracy(accuracies)
    
    # Sorting the accuracies
    accuracies.sort(reverse=True, key=lambda x: x[len(accuracies) - 1])
    print_parameters_accuracy(accuracies)
    
    return accuracies[0][0], accuracies[0][1], accuracies[0][2]

In [14]:
def get_data_ready(symbol_name, max_features, C_list, gamma_list, kernel):
    start_time = time.time()
#     num_features, C, gamma = find_optimal_values(max_features, C_list, gamma_list, kernel, 
#                                                  num_splits=10, symbol_name = symbol_name)
    num_features, C, gamma = 3, 100000000, 0.01
    end_time = time.time()
    print('Time taken for Cross Validation:', end_time - start_time)
    
    X_train, X_test, Y_train, Y_test = hm.prepare_data(num_features)
    X_train, X_test, Y_train, Y_test = X_train.values, X_test.values, Y_train.values, Y_test.values
    return X_train, X_test, Y_train, Y_test, C, gamma    
    
#     X_train, X_test, Y_train, Y_test = hm.prepare_data(2)
#     X_train, X_test, Y_train, Y_test = X_train.values, X_test.values, Y_train.values, Y_test.values
#     return X_train, X_test, Y_train, Y_test, 1.0, 0.25   

### 1. SkLearn SVM Classifier

In [9]:
def sklearn_SVM_forecast(X_train, X_test, Y_train, Y_test, gamma, C = 1.0, kernel = 'rbf'):
    print('SKLEARN INBUILT SVM')
    clf = svm.SVC(kernel = kernel, C = C, gamma=gamma)
    clf.fit(X_train, Y_train)
    print('In-Built SVM (Accuracy) score -- kernel =', kernel, '--', clf.score(X_test, Y_test), '\n')
    print('Number of SVs =', len(clf.support_vectors_))

### 2. Predicting using Implementation

In [10]:
def implemented_SVM_forecast(X_train, X_test, Y_train, Y_test, gamma, C = 1.0, kernel = 'rbf'):
    my_SVM = Support_Vector_Machine_Classifier(C = C, kernel = kernel, gamma = gamma)
    print('Implemenation Fit Starting...')
    my_SVM.fit(X_train, Y_train)
    
    print('IMPLEMENTATION') 
    Y_pred = my_SVM.predict(X_test)
    hm.accuracy_metrics(Y_test, Y_pred)

### Running SVM

In [11]:
def forecast(X_train, X_test, Y_train, Y_test, gamma, C = 1.0, kernel = 'rbf'):
    print('C:', C, '\t gamma:', gamma)
    sklearn_SVM_forecast(X_train, X_test, Y_train, Y_test, gamma, C, kernel = kernel)
    implemented_SVM_forecast(X_train, X_test, Y_train, Y_test, gamma, C, kernel)
    
# X_train, X_test, Y_train, Y_test = hm.prepare_data(2)
# X_train, X_test, Y_train, Y_test = X_train.values, X_test.values, Y_train.values, Y_test.values
# forecast(X_train, X_test, Y_train, Y_test, 0.25, 1, 'rbf')

http://m.svms.org/regression/TaCa01.pdf  
Sensitivity of SVMs to parameters  
Application of support vector machines in Financial time series forecasting  
Francis E.H. Tay ∗, Lijuan Cao

In [12]:
def run_SVM(symbol_name):
    max_features = 10
    kernel = 'rbf'
    C_list = [1, 10, 33, 55, 78, 100]
    gamma_list = [1/100, 1/75, 1/50, 1/25, 1/10, 1]
    
    X_train, X_test, Y_train, Y_test, C, gamma = get_data_ready(symbol_name, max_features, C_list, gamma_list, kernel)
    forecast(X_train, X_test, Y_train, Y_test, gamma, C, kernel)

In [16]:
run_SVM(symbol_name = 'AAPL')

Time taken for Cross Validation: 9.5367431640625e-07
C: 100000000 	 gamma: 0.01
SKLEARN INBUILT SVM
In-Built SVM (Accuracy) score -- kernel = rbf -- 0.5160550458715596 

Number of SVs = 2435
C used: 100000000
Implemenation Fit Starting...
     pcost       dcost       gap    pres   dres
 0:  3.9285e+13 -1.7176e+16  2e+16  2e-06  3e-06
 1: -2.6451e+10 -3.8674e+14  4e+14  1e-05  3e-06
 2: -1.9497e+11 -4.8278e+12  5e+12  5e-06  3e-06
 3: -2.0943e+11 -8.5594e+11  6e+11  4e-06  3e-06
 4: -3.0027e+11 -4.4043e+11  1e+11  5e-06  4e-06
 5: -3.7309e+11 -3.9954e+11  3e+10  2e-06  5e-06
 6: -3.7682e+11 -3.9652e+11  2e+10  5e-06  5e-06
 7: -3.7913e+11 -3.9449e+11  2e+10  7e-06  5e-06
 8: -3.8101e+11 -3.9271e+11  1e+10  1e-06  5e-06
 9: -3.8234e+11 -3.9136e+11  9e+09  2e-05  5e-06
10: -3.8291e+11 -3.9081e+11  8e+09  6e-06  5e-06
11: -3.8374e+11 -3.8998e+11  6e+09  1e-05  5e-06
12: -3.8452e+11 -3.8921e+11  5e+09  5e-06  5e-06
13: -3.8503e+11 -3.8872e+11  4e+09  3e-06  5e-06
14: -3.8559e+11 -3.8817e+11